In [1]:
import json
import pandas as pd
import gensim
import nltk
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
import numpy as np
import math

# Creating Corpus

In [2]:
market_data = pd.read_csv('data/market_data.csv')
with open('json/readable.json') as json_file:
    heads = json.load(json_file)

In [3]:
head_dates =list(heads.keys()) #gets the dates into a list
dates = [d for d in market_data['Date'] if d.replace("-","") in heads.keys()]
#dates = [d for d in head_dates if (d[0:4] + "-" + d[4:6] + "-" + d[6::]) in list(market_data["Date"])]
vix_close = [market_data.loc[i].values[4] for i in range(len(market_data)) if market_data['Date'][i] in dates]
dji_close = [market_data.loc[i].values[-3] for i in range(len(market_data)) if market_data['Date'][i] in dates]

In [4]:
corpus = ['. '.join(heads[d.replace("-","")]) for d in dates] #constructs each date of headlines as a document

In [6]:
corpus2 = [''.join([' . '.join(heads[dates[i].replace("-","")]),
                    " . ".join(heads[dates[i+1].replace("-","")]),
                    " . ".join(heads[dates[i+2].replace("-","")])])
           for i in range(len(dates)-3)] #for looking back 3 days to predict 4th

# Create tf-idf Scores for 'of Day Headlines'

In [51]:
vectorizer = TfidfVectorizer(min_df=0.01, max_df=0.2,  norm=None) #get vector and tdidf scores
tfidf_scores = vectorizer.fit_transform(corpus)

In [52]:
tfidf_matrix = tfidf_scores.toarray() #put it to a matrix
df_tfidf_scores = pd.DataFrame(tfidf_matrix, columns=vectorizer.get_feature_names())
df_tfidf_scores.head()

,000,08,100,101,11,12,13,14,15,150,...,youtube,yuan,yukos,yum,zealand,zero,zika,zimbabwe,zone,zurich
0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,3.477608,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.000000,3.239483,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,2.704055,0.000000,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


# Create tf-idf Scores for '3 Day Advance Headlines'

In [23]:
tfidf_scores2 = vectorizer.fit_transform(corpus2)
tfidf_matrix2 = tfidf_scores2.toarray() #put it to a matrix
df_tfidf_scores2 = pd.DataFrame(tfidf_matrix2, columns=vectorizer.get_feature_names())
df_tfidf_scores2.head()

,01,02,03,04,05,06,07,08,09,10000,...,zions,zombie,zombies,zones,zoo,zte,zuckerberg,zuma,zurich,zynga
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [25]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.kernel_ridge import KernelRidge

In [41]:
len(vix_close[3::])

5047

In [61]:
y = vix_close[3::] #first 3 days are dedicated to predicting 3rd day
X_train, X_test, y_train, y_test = train_test_split(tfidf_matrix2, y, test_size=0.3, random_state=0)
print("VIX:")
#VIX models
#r = LinearRegression().fit(X_train,y_train)
#pred = r.predict(X_test)
#print("LIN REG MSE: " + str(mean_squared_error(pred,y_test)))

vixclf = KernelRidge(alpha=1,kernel = "poly")
vixclf.fit(X_train, y_train)
vixpred =vixclf.predict(X_test)
print("R^2 Score: " + str(vixclf.score(X_test,y_test)))
print("R: " + str(math.sqrt(vixclf.score(X_test,y_test))))
print("MSE: " + str(mean_squared_error(vixpred,y_test)))
print("Sqrt(MSE): " + str(math.sqrt(mean_squared_error(vixpred,y_test))))

print("DJI:")
#DJI Models
y = dji_close[3::]
X_train, X_test, y_train, y_test = train_test_split(tfidf_matrix2, y, test_size=0.3, random_state=0)

#r = LinearRegression().fit(X_train,y_train)
#pred = r.predict(X_test)
#print("LIN REG MSE: " + str(mean_squared_error(pred,y_test)))

djiclf = KernelRidge(alpha=1,kernel = "poly")
djiclf.fit(X_train, y_train)
djipred =djiclf.predict(X_test)
print("R^2 Score: " + str(djiclf.score(X_test,y_test)))
print("R: " + str(math.sqrt(djiclf.score(X_test,y_test))))
print("MSE: " + str(mean_squared_error(djipred,y_test)))
print("Sqrt(MSE): " + str(math.sqrt(mean_squared_error(djipred,y_test))))

VIX:
R^2 Score: 0.6942673980667708
R: 0.8332270987352552
MSE: 22.115089221190125
Sqrt(MSE): 4.702668308650964
DJI:
R^2 Score: 0.8380918954568944
R: 0.9154735908025389
MSE: 4460144.074145101
Sqrt(MSE): 2111.9053184612944


In [67]:
import pyperclip
arr = [str(d) for d in list(y_test)]
pyperclip.copy(','.join(arr))


# Modeling

In [185]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error


reg = LinearRegression().fit(X_train,y_train)
pred = reg.predict(X_test)
print(mean_squared_error(pred,y_test))

3.180238643390024e+29


## Kernel Ridge with VIX

In [54]:
y = vix_close
X_train, X_test, y_train, y_test = train_test_split(tfidf_matrix, y, test_size=0.3, random_state=0)

clf = KernelRidge(alpha=1,kernel = "poly")
clf.fit(X_train, y_train)
pred =clf.predict(X_test)
print("R^2 Score: " + str(clf.score(X_test,y_test)))
print("R : " + str(math.sqrt(clf.score(X_test,y_test))))
print("MSE: " + str(mean_squared_error(pred,y_test)))

R^2 Score: 0.3671007717845833
R : 0.605888415291614
MSE: 45.99317472403121


## Kernel Ridge with DJI

In [68]:
y = dji_close
X_train, X_test, y_train, y_test = train_test_split(tfidf_matrix, y, test_size=0.3, random_state=0)

clf = KernelRidge(alpha=1,kernel = "poly")
clf.fit(X_train, y_train)
pred =clf.predict(X_test)
print("R^2 Score: " + str(clf.score(X_test,y_test)))
print("R : " + str(math.sqrt(clf.score(X_test,y_test))))
print("MSE: " + str(mean_squared_error(pred,y_test)))
print("Sqrt(MSE): " + str(math.sqrt(mean_squared_error(pred,y_test))))

R^2 Score: 0.6956996617893683
R : 0.834086123724264
MSE: 8295060.98485426
Sqrt(MSE): 2880.1147520288596


In [70]:
arr = [str(d) for d in list(y_test)]
pyperclip.copy(','.join(arr))

# Testing

In [73]:
[" . ".join(heads[dates[i+k].replace("-","")]) for k in range(5)]

NameError: name 'i' is not defined

In [78]:
for j in range(1,6):
    corpus3 = [''.join([" . ".join(heads[dates[i+k].replace("-","")]) for k in range(j)])
           for i in range(len(dates)-j)]
    tfidf_scores3 = vectorizer.fit_transform(corpus3)
    tfidf_matrix3 = tfidf_scores3.toarray() #put it to a matrix
    
    y = vix_close[j::] #first 3 days are dedicated to predicting jth day
    X_train, X_test, y_train, y_test = train_test_split(tfidf_matrix3, y, test_size=0.3, random_state=0)
    vixclf = KernelRidge(alpha=1,kernel = "poly")
    vixclf.fit(X_train, y_train)
    vixpred =vixclf.predict(X_test)
    print("VIX: " + str(j))
    print("R^2 Score: " + str(vixclf.score(X_test,y_test)))
    print("R: " + str(math.sqrt(vixclf.score(X_test,y_test))))
    print("MSE: " + str(mean_squared_error(vixpred,y_test)))
    print("Sqrt(MSE): " + str(math.sqrt(mean_squared_error(vixpred,y_test))))
    print("---------------------------")
     
    y = dji_close[j::]
    X_train, X_test, y_train, y_test = train_test_split(tfidf_matrix3, y, test_size=0.3, random_state=0)
    djiclf = KernelRidge(alpha=1,kernel = "poly")
    djiclf.fit(X_train, y_train)
    djipred =djiclf.predict(X_test)
    print("DJI: " + str(j))
    print("R^2 Score: " + str(djiclf.score(X_test,y_test)))
    print("R: " + str(math.sqrt(djiclf.score(X_test,y_test))))
    print("MSE: " + str(mean_squared_error(djipred,y_test)))
    print("Sqrt(MSE): " + str(math.sqrt(mean_squared_error(djipred,y_test))))
    
    print("********************")

VIX: 1
R^2 Score: 0.36620556227529255
R: 0.6051492066220466
MSE: 44.40059344289661
Sqrt(MSE): 6.663377029922335
---------------------------
DJI: 1
R^2 Score: 0.6843750702408062
R: 0.8272696478421085
MSE: 8872092.104635814
Sqrt(MSE): 2978.6057316529514
********************
VIX: 2
R^2 Score: 0.5887525447137796
R: 0.7673021208844529
MSE: 29.258880736802073
Sqrt(MSE): 5.409147875294414
---------------------------
DJI: 2
R^2 Score: 0.7733603898480989
R: 0.8794091140351564
MSE: 6211646.302334965
Sqrt(MSE): 2492.317456171056
********************
VIX: 3
R^2 Score: 0.6942673980667708
R: 0.8332270987352552
MSE: 22.115089221190125
Sqrt(MSE): 4.702668308650964
---------------------------
DJI: 3
R^2 Score: 0.8380918954568944
R: 0.9154735908025389
MSE: 4460144.074145101
Sqrt(MSE): 2111.9053184612944
********************
VIX: 4
R^2 Score: 0.7409685332117526
R: 0.8607952911184823
MSE: 18.46195807652712
Sqrt(MSE): 4.296738073996031
---------------------------
DJI: 4
R^2 Score: 0.8856363105277475
R: 0.9

In [81]:
import pickle
file = 'dji.sav'
pickle.dump(djiclf, open(file, 'wb'))

In [83]:
loaded_model = pickle.load(open(file, 'rb'))
result = loaded_model.score(X_test, y_test)
print(result)
print(math.sqrt(result))

0.9098779712104507
0.9538752388077021
